In [1]:
import pandas as pd
import numpy as np
from numpy import linalg

# Data

In [2]:
data = pd.read_csv('data.csv')
data

,u,v,X,Y,Z
0,880,214,312.747,309.140,30.086
1,43,203,305.796,311.649,30.356
2,270,197,307.694,312.358,30.418
3,886,347,310.149,307.186,29.298
4,745,302,311.937,310.105,29.216
5,943,128,311.202,307.572,30.682
6,476,590,307.106,306.876,28.660
7,419,214,309.317,312.490,30.230
8,317,335,307.435,310.151,29.318
9,783,521,308.253,306.300,28.881


# SVD Method

In [3]:
n, m = len(data), 12
A = np.zeros((2*n, m))
A[:n, 0:3] = data[['X', 'Y', 'Z']]
A[:n, 3] = 1.
A[:n, 8:11] = -data['u'].to_numpy().reshape(-1, 1) * data[['X', 'Y', 'Z']]
A[:n, 11] = -data['u']

A[n:, 4:7] = data[['X', 'Y', 'Z']]
A[n:, 7] = 1.
A[n:, 8:11] = -data['v'].to_numpy().reshape(-1, 1) * data[['X', 'Y', 'Z']]
A[n:, 11] = -data['v']

U, D, V = linalg.svd(A)
p_svd = V[-1]
P_svd = p_svd.reshape(3, 4)

In [4]:
print("- Matric P using SVD Method -")
print(P_svd)

- Matric P using SVD Method -
[[-3.09963996e-03 -1.46204548e-04  4.48497465e-04  9.78930678e-01]
 [-3.07018252e-04 -6.37193664e-04  2.77356178e-03  2.04144405e-01]
 [-1.67933533e-06 -2.74767684e-06  6.83964827e-07  1.32882928e-03]]


# Pseudo Inverse Method

In [5]:
n, m = len(data), 11
A = np.zeros((2*n, m))
b = np.zeros(2*n)
A[:n, 0:3] = data[['X', 'Y', 'Z']]
A[:n, 3] = 1.
A[:n, 8:11] = -data['u'].to_numpy().reshape(-1, 1) * data[['X', 'Y', 'Z']]
b[:n] = data['u']

A[n:, 4:7] = data[['X', 'Y', 'Z']]
A[n:, 7] = 1.
A[n:, 8:11] = -data['v'].to_numpy().reshape(-1, 1) * data[['X', 'Y', 'Z']]
b[n:] = data['v']

A_plus = np.matmul(linalg.inv(np.matmul(A.T, A)), A.T)
q = np.matmul(A_plus, b)
p_pim = np.hstack([q, 1])
P_pim = p_pim.reshape(3, 4)

In [6]:
print("- Matric P using Pseudo Inverse Method -")
print(P_pim)

- Matric P using Pseudo Inverse Method -
[[-2.33259099e+00 -1.09993074e-01  3.37413843e-01  7.36673912e+02]
 [-2.31050254e-01 -4.79506022e-01  2.08717636e+00  1.53627753e+02]
 [-1.26379607e-03 -2.06770916e-03  5.14635179e-04  1.00000000e+00]]


# Evaluation

In [7]:
XYZ1 = np.hstack([data[['X', 'Y', 'Z']].to_numpy(), np.ones((n, 1))])
df = pd.DataFrame()

uv = np.matmul(XYZ1, P_svd.T)
uv = uv[:, :2] / uv[:, -1:]
df['svd_u'] = uv[:, 0]
df['svd_v'] = uv[:, 1]

uv = np.matmul(XYZ1, P_pim.T)
uv = uv[:, :2] / uv[:, -1:]
df['pim_u'] = uv[:, 0]
df['pim_v'] = uv[:, 1]

df['real_u'] = data['u']
df['real_v'] = data['v']

df

,svd_u,svd_v,pim_u,pim_v,real_u,real_v
0,879.435317,214.590586,879.432494,214.590473,880,214
1,43.289025,203.775431,43.289681,203.775025,43,203
2,269.694194,196.858653,269.694756,196.858345,270,197
3,885.640835,346.615978,885.643653,346.615045,886,347
4,745.196222,302.197516,745.192787,302.196688,745,302
5,943.305585,127.597382,943.307870,127.598864,943,128
6,476.274123,589.799211,476.281230,589.803546,476,590
7,419.075344,213.333377,419.074586,213.333068,419,214
8,316.518090,334.434324,316.517657,334.434389,317,335
9,783.113563,520.458968,783.124371,520.459984,783,521


In [8]:
u_error = np.mean(np.abs((df['real_u'] - df['svd_u']) / df['real_u']))
v_error = np.mean(np.abs((df['real_v'] - df['svd_v']) / df['real_v']))
print(f"- Relative error of SVD method -")
print(f"u: {u_error*100:.4f}%")
print(f"v: {v_error*100:.4f}%")
print()

u_error = np.mean(np.abs((df['real_u'] - df['pim_u']) / df['real_u']))
v_error = np.mean(np.abs((df['real_v'] - df['pim_v']) / df['real_v']))
print(f"- Relative error of Pseudo Inverse method -")
print(f"u: {u_error*100:.4f}%")
print(f"v: {v_error*100:.4f}%")
print()

- Relative error of SVD method -
u: 0.1267%
v: 0.1669%

- Relative error of Pseudo Inverse method -
u: 0.1269%
v: 0.1668%

